In [1]:
%load_ext lab_black

# Find a common destination from two different airports from historical traffic data

#### From two European airports, with direct flights, whatever the company

In [2]:
import pandas as pd
from datetime import datetime, timedelta

#### <br> Example with a trip from Toulouse, France (LFBO) and Amsterdam, Netherlands (EHAM), end of November

In [3]:
first_airport = "LFBO"
second_airport = "EHAM"

#### <br> Load airports and airlines table from openflights.org

In [4]:
airports_url = (
    "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"
)
airlines_url = (
    "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat"
)

airports_df = pd.read_csv(
    airports_url,
    header=None,
    names=[
        "id",
        "name",
        "city",
        "country",
        "iata",
        "icao",
        "lat",
        "lon",
        "alt",
        "timezone",
        "dst",
        "tz",
        "type",
        "source",
    ],
)
airlines_df = pd.read_csv(
    airlines_url,
    header=None,
    names=["id", "name", "alias", "iata", "icao", "callsign", "country", "active"],
)

In [5]:
def get_city(airport_icao):
    ans = airport_icao
    if not airports_df.query("icao == @airport_icao").empty:
        df = airports_df.query("icao == @airport_icao")
        ans = df.city.item() + "-" + df.name.item()
    return ans

In [6]:
def get_airline(airline_icao):
    ans = airline_icao
    if not airlines_df.query("icao == @airline_icao").empty:
        ans = airlines_df.query("icao == @airline_icao").name.item()
    return ans

#### <br> Load the so6 reference traffic file (e.g. one year before) as a pandas dataframe

In [7]:
my_so6_traffic_file = "my_so6_traffic_file.so6"
columns = [
    "segment_identifier",
    "flight_origin",
    "flight_destination",
    "aircraft_type",
    "time_begin",
    "time_end",
    "fl_begin",
    "fl_end",
    "status",
    "callsign",
    "date_begin",
    "date_end",
    "lat_begin",
    "lon_begin",
    "lat_end",
    "lon_end",
    "flight_identifier",
    "sequence",
    "length",
    "parity",
]

parser = lambda x, y: pd.datetime.strptime(x + y, "%y%m%d%H%M%S")

df = pd.read_csv(
    my_so6_traffic_file,
    sep=" ",
    header=None,
    names=columns,
    parse_dates={
        "datetime_begin": ["date_begin", "time_begin"],
        "datetime_end": ["date_end", "time_end"],
    },
    date_parser=parser,
)

coord_cols = ["lat_begin", "lon_begin", "lat_end", "lon_end"]
df[coord_cols] = df[coord_cols].apply(lambda x: x / 60)

t = df[
    [
        "callsign",
        "datetime_begin",
        "datetime_end",
        "flight_origin",
        "flight_destination",
        "fl_begin",
        "fl_end",
        "lat_begin",
        "lat_end",
        "lon_begin",
        "lon_end",
        "aircraft_type",
        "length",
        "flight_identifier",
    ]
]
t = t.query("flight_origin == @first_airport or flight_origin == @second_airport")
t = t.query("length > 0")  # filter null segments
t.sample(5)

,callsign,datetime_begin,datetime_end,flight_origin,flight_destination,fl_begin,fl_end,lat_begin,lat_end,lon_begin,lon_end,aircraft_type,length,flight_identifier
24955,DLH7KY,2018-11-23 10:57:32,2018-11-23 10:58:32,EHAM,EDDM,301,279,49.383611,49.293889,9.824722,9.978889,A319,8.081272,225202876
169989,KLM81K,2018-11-23 09:09:30,2018-11-23 09:09:42,EHAM,LIRF,142,150,52.176944,52.161389,5.411111,5.446667,B737,1.607213,225200717
1002013,AFR82QZ,2018-11-23 10:09:07,2018-11-23 10:09:35,LFBO,LFPG,25,35,43.606111,43.598889,1.392778,1.399167,A320,0.514619,225202353
1172683,TAY054G,2018-11-22 23:59:34,2018-11-23 00:00:06,LFBO,EBLG,201,192,50.374167,50.389722,3.968056,4.049167,B734,3.240625,225193071
1043954,JAI233,2018-11-23 10:59:50,2018-11-23 11:00:58,EHAM,VIDP,256,271,52.047778,52.023611,6.535000,6.764167,B77W,8.581972,225203591


#### <br> Identify common destination and associated airlines

In [8]:
for dest in t.query("flight_origin == @first_airport").flight_destination.unique():
    second_table = t.query(
        "flight_origin == @second_airport and flight_destination == @dest"
    )
    if len(second_table.index) > 0:
        print(f"\n -> Destination {get_city(dest)}")
        first_table = t.query(
            "flight_origin == @first_airport and flight_destination == @dest"
        )
        print(f"from {get_city(first_airport)} with airlines:")
        print(set([get_airline(cs[:3]) for cs in first_table.callsign.unique()]))
        print(f"from {get_city(second_airport)} with airlines:")
        print(set([get_airline(cs[:3]) for cs in second_table.callsign.unique()]))

/Users/tdubot/Documents/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: `item` has been deprecated and will be removed in a future version
  """
/Users/tdubot/Documents/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: `item` has been deprecated and will be removed in a future version
  after removing the cwd from sys.path.



 -> Destination Villafranca-Verona Villafranca Airport
from Toulouse-Toulouse-Blagnac Airport with airlines:
{'VOLOTEA Airways'}
from Amsterdam-Amsterdam Airport Schiphol with airlines:
{'Transavia Holland'}

 -> Destination Strasbourg-Strasbourg Airport
from Toulouse-Toulouse-Blagnac Airport with airlines:
{'VOLOTEA Airways', 'HOP'}
from Amsterdam-Amsterdam Airport Schiphol with airlines:
{'Air France'}

 -> Destination Marseille-Marseille Provence Airport
from Toulouse-Toulouse-Blagnac Airport with airlines:
{'HOP'}
from Amsterdam-Amsterdam Airport Schiphol with airlines:
{'KLM Royal Dutch Airlines'}

 -> Destination Rennes-Rennes-Saint-Jacques Airport
from Toulouse-Toulouse-Blagnac Airport with airlines:
{'HOP'}
from Amsterdam-Amsterdam Airport Schiphol with airlines:
{'Air France'}

 -> Destination Geneva-Geneva Cointrin International Airport
from Toulouse-Toulouse-Blagnac Airport with airlines:
{'EES', 'easyJet'}
from Amsterdam-Amsterdam Airport Schiphol with airlines:
{'EZS', 'K